In [2]:
#%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.quantum_info import *
import numpy as np


    
# Loading your IBM Q account(s)
#provider = IBMQ.load_account()

#contants
n = 1
theta = 1

#create quantum circuit
qc = QuantumCircuit(4)

#create Quantum operators

F1_dag = Operator([
    [1, 0, 0, 0],
    [0, 1/np.sqrt(2), 1/np.sqrt(2), 0],
    [0, 1/np.sqrt(2)*np.exp(-2*np.pi*1j), -1/np.sqrt(2)*np.exp(-2*np.pi*1j), 0],
    [0, 0, 0, -np.exp(-2*np.pi*1j)]])

F0_dag = Operator([
    [1, 0, 0, 0],
    [0, 1/np.sqrt(2), 1/np.sqrt(2), 0],
    [0, 1/np.sqrt(2), -1/np.sqrt(2), 0],
    [0, 0, 0, -1]])
B1_dag = Operator([
    [np.cos(theta/2), 0, 0, 1j*np.sin(theta/2)],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [ 1j*np.sin(theta/2),0,0,np.cos(theta/2)]])

fswap = Operator([
        [1,0,0,0],
        [0,0,1,0],
        [0,1,0,0],
        [0,0,0,-1]
])



#Create quantum Gates
qc.unitary(B1_dag, [0, 1], label='B1_dag')
qc.unitary(F1_dag, [0, 1], label='F1_dag')
qc.unitary(F0_dag, [2, 3], label='F0_dag')
qc.unitary(fswap,[1,2],label ='fSWAP')
qc.unitary(F0_dag, [0, 1], label='F0_dag')
qc.unitary(F0_dag, [2, 3], label='F0_dag')
qc.unitary(fswap,[1,2],label ='fSWAP')

#display circuit
qc.draw()


┌─────────┐┌─────────┐          ┌─────────┐          
q_0: ┤0        ├┤0        ├──────────┤0        ├──────────
     │  B1_dag ││  F1_dag │┌────────┐│  F0_dag │┌────────┐
q_1: ┤1        ├┤1        ├┤0       ├┤1        ├┤0       ├
     ├─────────┤└─────────┘│  fSWAP │├─────────┤│  fSWAP │
q_2: ┤0        ├───────────┤1       ├┤0        ├┤1       ├
     │  F0_dag │           └────────┘│  F0_dag │└────────┘
q_3: ┤1        ├─────────────────────┤1        ├──────────
     └─────────┘                     └─────────┘

$ \newcommand{\ket}[1]{\left|{#1}\right\rangle}$
$ \newcommand{\bra}[1]{\left\langle{#1}\right|}$
$$ \newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

when it comes to implementation:  
first try to run circuit on simulator 
second try to run on real hardware

long term goal is reproduce figures 5 (Ground state of a spin chain)and 6 (time evolution of operator $\sigma_z$) 

short term goal is initialize the state to  $ \ket{0000}  $ and $\ket{ 1100} $